# Oscillation Presence

Topic: periodic activity may or may not actually be present

Sim: demonstrate signals with and without periodic activity.

- FOOOF rhythmicity example
    - https://fooof-tools.github.io/fooof/auto_motivations/concepts/plot_DoYouEvenOscillate.htm
- FOOOF filter example
    - https://fooof-tools.github.io/fooof/auto_motivations/concepts/plot_IfYouFilterTheyWillCome.html
- Also related to FOOOF paper Fig S1.

Core citations / links:

- FOOOF